In [1]:
%pip install pymorphy3
import re
import os
import pymorphy3

morph = pymorphy3.MorphAnalyzer(lang='ru')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


1. Определим рабочую директорию и сгруппируем тексты по автору в новые файлы для удобства анализа на следующий этапах.

In [2]:
# Проверяем рабочую директорию
print(os.getcwd())

path = r"C:\Users\1610043\Desktop\Проектик"
file_list = os.listdir(path)

#Создадим словарь для хранения текстов, сгруппированных по авторам
authors_texts = {}

#Переберем все файлы
for filename in file_list:
    if filename.endswith(".txt") and "_merged" not in filename:
        parts = filename.split(".")
        author = parts[1]
        with open(filename, "r", encoding="utf-8") as file:
            text = file.read()
        if author not in authors_texts:
            authors_texts[author] = text
        else:
            authors_texts[author] += " " + text

c:\Users\1610043\Desktop\Проектик


In [4]:
#ИИ подсказал, как пересохранить объединенные тексты в новые файлы
# Сохраняем объединённые тексты в новые файлы
for author, full_text in authors_texts.items():
    new_filename = f"{author}_merged.txt"
    with open(new_filename, "w", encoding="utf-8") as file:
        file.write(full_text)
    print(f"Сохранено: {new_filename}") #шаг нужен, чтобы понять, какие файлы пересохранились

Сохранено: Иванов_merged.txt
Сохранено: Сальников_merged.txt
Сохранено: Пелевин_merged.txt
Сохранено: Сорокин_merged.txt
Сохранено: Яхина_merged.txt
Сохранено: Водолазкин_merged.txt
Сохранено: Прилепин_merged.txt
Сохранено: Елизаров_merged.txt


2. Приступим к очистке текстов и поиску необходимых единиц в текстах (цветообозначений, словосочетакния сущ + цветобозначение в любом порядке)

In [3]:
#Список цветообозначений для поиска (12 базовых для русского языка в орфографически корректной форме)
COLOR_WORDS = {
    'красный', 'оранжевый', 'жёлтый', 'зелёный', 'синий', 'голубой',
    'фиолетовый','чёрный', 'белый', 'серый', 'розовый', 'коричневый'}

2.1 Напишем функции, которые помогут выполнить планируемые шаги:

In [4]:
def clean_text(text):
    """Функция очищает текст: приводит к нижнему регистру, удаляет лишний "шум", сжимает пробелы."""
    text = text.lower()
    text = re.sub(r'\b[IVXLCDM]+\b', '', text)  # римские цифры (они есть в ряде текстов)
    text = re.sub(r'[\(\)\[\]\{\}]', '', text)  # скобки (разные виды скобок встречаются в текстах)
    text = re.sub(r'\s+', ' ', text)  # сжимаем пробелы (есть пробелы между главами, нужно убрать)
    text = re.sub(r'[^а-яё\s\-]', '', text)  # оставляем только буквы, пробелы, дефисы
    return text.strip() 

In [5]:
#ИИ помог с функцией, чтобы была возможность все желаемое сделать сразу (и собрать леммы цветообозначений, и проверить пары сущ + цвет, а также собрать их)
def find_color_noun_pairs_and_colors(text):
    """Функция:
    1. Находит все цветообозначения (с лемматизацией).
    2. Ищет пары «цвет + сущ» и «сущ + цвет» в окне ±4 слова с проверкой согласования по морф. признакам.
    3. Возвращает:
      - список пар (исходная форма цвета, исходная форма сущ из текста);
      - список всех цветообозначений (в форме лемм).
    """
    words = text.split()
    pairs = []
    all_colors = []

    for i, word in enumerate(words):
        # Разбор морфологических признаков слова
        parsed = morph.parse(word)[0]
        lemma = parsed.normal_form
        # Проверяем, является ли слово цветообозначением
        if lemma not in COLOR_WORDS:
            continue
        # Сохраняем лемму в общий список цветообозначений
        all_colors.append(lemma)
        # Сохраняем исходную форму цветообозначения для проверки согласования
        color_info = {
            'gender': parsed.tag.gender,
            'number': parsed.tag.number,
            'case': parsed.tag.case,
            'original': word
        }
        # Поиск существительного справа от цвета (цвет + сущ)
        for j in range(i + 1, min(len(words), i + 5)):
            next_parsed = morph.parse(words[j])[0]
            if next_parsed.tag.POS == 'NOUN':
                noun_info = {
                    'gender': next_parsed.tag.gender,
                    'number': next_parsed.tag.number,
                    'case': next_parsed.tag.case,
                    'original': words[j]
                }
                # Проверка согласования по роду, числу и падежу
                if (color_info['gender'] == noun_info['gender'] and
                    color_info['number'] == noun_info['number'] and
                    color_info['case'] == noun_info['case']):
                    pairs.append((color_info['original'], noun_info['original']))
                    break  # нашли первое подходящее — переходим к следующему слову

        # Поиск существительного слева от цвета (сущ + цвет)
        for j in range(i - 1, max(-1, i - 5), -1):
            prev_parsed = morph.parse(words[j])[0]
            if prev_parsed.tag.POS == 'NOUN':
                noun_info = {
                    'gender': prev_parsed.tag.gender,
                    'number': prev_parsed.tag.number,
                    'case': prev_parsed.tag.case,
                    'original': words[j]
                }
                # Проверка согласования (принцип, как и при поиске "справа")
                if (color_info['gender'] == noun_info['gender'] and
                    color_info['number'] == noun_info['number'] and
                    color_info['case'] == noun_info['case']):
                    pairs.append((color_info['original'], noun_info['original']))  # порядок: цвет + сущ
                    break 

    return pairs, all_colors


2.2 Пройдемся по всем текстам: очистим, лемматизируем цветообозначения (+ добавим в отдельные файлы по автору), выделим пары цвет-ние + сущ-е

In [6]:
for filename in os.listdir(path):
    if filename.endswith("_merged.txt"):
        author = filename.replace("_merged.txt", "")#получаем фамилию автора из названия файла

        with open(os.path.join(path, filename), "r", encoding="utf-8") as file:
            text = file.read()

        cleaned_text = clean_text(text)
        color_noun_pairs, all_colors = find_color_noun_pairs_and_colors(cleaned_text)

        # Сохраняем очищенный текст
        with open(os.path.join(path, f"{author}_cleaned.txt"), "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        # Сохраняем пары «цвет + сущ» (каждая пара на новой строке)
        with open(os.path.join(path, f"{author}_color_noun_pairs.txt"), "w", encoding="utf-8") as f:
            for color, noun in color_noun_pairs:
                f.write(f"{color} {noun}\n")

        # Сохраняем только цветообозначения (по одному на строку)
        with open(os.path.join(path, f"{author}_colors_only.txt"), "w", encoding="utf-8") as f:
            for color in all_colors:
                f.write(f"{color}\n")

        # Выводим статистику для контроля
        print(f"Обработан автор: {author}")
        print(f"  - Найдено сочетаний «цвет+сущ»: {len(color_noun_pairs)}")
        if color_noun_pairs:
            print(f"  - Примеры пар: {color_noun_pairs[:2]}")  # первые 2 пары
        print(f"  - Найдено цветообозначений: {len(all_colors)}")
        if all_colors:
            print(f"  - Примеры цветов: {all_colors[:5]}")  # первые 5 цветов
        print(f"  - Файлы сохранены: "
              f"{author}_cleaned.txt, "
              f"{author}_color_noun_pairs.txt, "
              f"{author}_colors_only.txt\n")

Обработан автор: Водолазкин
  - Найдено сочетаний «цвет+сущ»: 242
  - Примеры пар: [('белом', 'халате'), ('белая', 'косынка')]
  - Найдено цветообозначений: 711
  - Примеры цветов: ['белый', 'белый', 'белый', 'розовый', 'белый']
  - Файлы сохранены: Водолазкин_cleaned.txt, Водолазкин_color_noun_pairs.txt, Водолазкин_colors_only.txt

Обработан автор: Елизаров
  - Найдено сочетаний «цвет+сущ»: 654
  - Примеры пар: [('белая', 'лилия'), ('серого', 'цвета')]
  - Найдено цветообозначений: 1319
  - Примеры цветов: ['белый', 'серый', 'чёрный', 'синий', 'чёрный']
  - Файлы сохранены: Елизаров_cleaned.txt, Елизаров_color_noun_pairs.txt, Елизаров_colors_only.txt

Обработан автор: Иванов
  - Найдено сочетаний «цвет+сущ»: 785
  - Примеры пар: [('зелёная', 'футболка'), ('синее', 'небо')]
  - Найдено цветообозначений: 1644
  - Примеры цветов: ['зелёный', 'синий', 'зелёный', 'серый', 'серый']
  - Файлы сохранены: Иванов_cleaned.txt, Иванов_color_noun_pairs.txt, Иванов_colors_only.txt

Обработан автор: